In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRITON_PTXAS_PATH"] = "/usr/local/cuda/bin/ptxas"
import re
import time
import random
import warnings
from collections import Counter
import numpy as np, pandas as pd

import torch
import vllm
from vllm import LLM, SamplingParams


warnings.simplefilter('ignore')
print('PyTorch version:', torch.__version__)
print('vLLM:', vllm.__version__)

/workspace/open-r1/openr1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 03-28 22:14:06 __init__.py:183] Automatically detected platform cuda.


2025-03-28 22:14:07,213	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


PyTorch version: 2.5.1+cu124
vLLM: 0.7.1


In [5]:
# We want to compare the logits of the regular 7b model with the logits of the 7b model that's been trained with keep tags.

keep_trained_model_id = "samitizerxu/DS-7B-Qwen-distil-DPO-keep"
regular_model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
seed_everything(seed=0)

start_time = time.time()
cutoff_time = start_time + (4 * 60 + 45) * 60
cutoff_times = [int(x) for x in np.linspace(cutoff_time, start_time + 60 * 60, 50 + 1)]



MAX_NUM_SEQS = 32
MAX_MODEL_LEN = 8192 * 3 // 2

llm_keep = LLM(
    "samitizerxu/DS-7B-Qwen-distil-DPO-keep",
#    dtype="half",                 # The data type for the model weights and activations
    max_num_seqs=MAX_NUM_SEQS,    # Maximum number of sequences per iteration. Default is 256
    max_model_len=MAX_MODEL_LEN,  # Model context length
    trust_remote_code=True,       # Trust remote code (e.g., from HuggingFace) when downloading the model and tokenizer
    tensor_parallel_size=1,       # The number of GPUs to use for distributed execution with tensor parallelism
    gpu_memory_utilization=0.47,  # The ratio (between 0 and 1) of GPU memory to reserve for the model
    seed=2024,
)

tokenizer = llm_keep.get_tokenizer()

INFO 03-28 22:17:11 config.py:526] This model supports multiple tasks: {'reward', 'generate', 'embed', 'classify', 'score'}. Defaulting to 'generate'.
INFO 03-28 22:17:11 llm_engine.py:232] Initializing a V0 LLM engine (v0.7.1) with config: model='samitizerxu/DS-7B-Qwen-distil-DPO-keep', speculative_config=None, tokenizer='samitizerxu/DS-7B-Qwen-distil-DPO-keep', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=12288, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=2024, served_model_name=samit

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.91it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.11it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.60it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.41it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.60it/s]



INFO 03-28 22:17:16 model_runner.py:1116] Loading model weights took 14.2656 GB
INFO 03-28 22:17:17 worker.py:266] Memory profiling takes 0.78 seconds
INFO 03-28 22:17:17 worker.py:266] the current vLLM instance can use total_gpu_memory (79.20GiB) x gpu_memory_utilization (0.47) = 37.23GiB
INFO 03-28 22:17:17 worker.py:266] model weights take 14.27GiB; non_torch_memory takes 0.16GiB; PyTorch activation peak memory takes 1.66GiB; the rest of the memory reserved for KV Cache is 21.14GiB.
INFO 03-28 22:17:17 executor_base.py:108] # CUDA blocks: 24744, # CPU blocks: 4681
INFO 03-28 22:17:17 executor_base.py:113] Maximum concurrency for 12288 tokens per request: 32.22x
INFO 03-28 22:17:21 model_runner.py:1435] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory

Capturing CUDA graph shapes: 100%|██████████| 7/7 [00:04<00:00,  1.47it/s]

INFO 03-28 22:17:25 model_runner.py:1563] Graph capturing finished in 5 secs, took 0.15 GiB
INFO 03-28 22:17:25 llm_engine.py:429] init engine (profile, create kv cache, warmup model) took 9.08 seconds


In [18]:
max_tokens = 8196
sampling_params = SamplingParams(
    temperature=0.6,               # Randomness of the sampling
    top_p=0.90,                    # Cumulative probability of the top tokens to consider
    min_p=0.05,                    # Minimum probability for a token to be considered
    skip_special_tokens=False,      # Whether to skip special tokens in the output
    max_tokens=max_tokens,         # Maximum number of tokens to generate
    stop=["</think>"],             # List of strings that stop the generation
    seed=777,
    # logit_bias={
    #     151665: 40.0,
    #     151666: 40.0,
    # }
)

system_prompt = "You are a helpful AI Assistant who provides well-reasoned and accurate responses to math problems. You should think step-by-step, and the answer is an integer. You should wrap important steps in <keep> .. </keep> tags. Return the final answer within \\boxed{} after taking modulo 1000."

user_prompt = "Let $N$ denote the number of ordered triples of positive integers $(a, b, c)$ such that $a, b, c \leq 3^6$ and $a^3 + b^3 + c^3$ is a multiple of $3^7$. Find the remainder when $N$ is divided by $1000$."

prompt = tokenizer.apply_chat_template(
    [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    tokenize=False,
    add_generation_prompt=True
)

response = llm_keep.generate(
    prompts=[prompt+'<keep>'],
    sampling_params=sampling_params,
)

print(response[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [01:03<00:00, 63.28s/it, est. speed input: 2.29 toks/s, output: 129.54 toks/s]

Okay, so I need to find the number of ordered triples (a, b, c) where each of a, b, c is a positive integer less than or equal to 3^6. Additionally, the sum of their cubes, a³ + b³ + c³, must be divisible by 3^7. Then, I have to find the remainder when this number, N, is divided by 1000. Hmm, let me try to break this down step by step.

First, let's note that 3^6 is 729, so each of a, b, c can be any integer from 1 to 729. But we're dealing with modulo 3^7, which is 2187, since 3^7 is 2187. So, the problem is about the sum of cubes modulo 2187.

I remember that when dealing with modular equations, especially with exponents, it's often helpful to use modular arithmetic properties and maybe some number theory concepts. Since 3 is a prime, I might consider using the concept of lifting the exponent or perhaps looking at the problem modulo 3^k for increasing k.

Wait, but 3^7 is 2187, and 3^6 is 729, so 3^7 is just 3 times 3^6. So, 3^7 is 2187, which is a bit larger than 729, so 729 is 3^6,